# 第二十八讲：$k$-均值聚类（$k$-means clustering）
___

在介绍算法之前，让我们来看看什么是聚类：

> 对于大量未给定标签的数据集，聚类就是根据数据的内在相似性（距离）将数据集划分为多个类别的方法。聚类的目的是使得类内数据的相似性大，而类间的数据相似性低。

用数学描述来看，就是给定了$\{\mathbf{x_i}\}, i=1,\ldots,N$，将其划分为$K$个类别$C_k=\{\mathbf{x_j}, \ldots\}$，使得$\sum_{k=1}^K \operatorname{sim}(C_k)$最大化，而最小化$\sum_{i=1}^{K-1} \sum_{j=i+1}^K \operatorname{sim}(C_i, C_j)$。

聚类是一种非监督的学习方法（unsupervised learning）。

## 1. 迭代的思想（Iterative approach）

对于给定的样本数据和类别数目$k$，
- 首先给出初始划分
- 通过迭代改变样本和簇的隶属关系，使得每一次改进之后的划分方案都较前一次更佳。

$k$-means算法执行的正是这样一种迭代的思路。

这是一种交替更新（alternating updating）的方法，在机器学习中是非常常用的一种思路。

## 2. $k$-means算法

也被称为$k$-均值算法，是一种广泛使用的聚类算法，同时还是其他聚类算法的基础（如谱聚类）。

这是基本的__算法步骤__：

1. 初始化$k$个聚类中心$\mathbf{u_1, \ldots, u_k}$

2. 更新每个样本的聚类隶属关系：
$$
y_i = \operatorname{arg min}_{1 \le j \le k} \operatorname{dist}(\mathbf{x}_i, \mathbf{u}_j)
$$

3. 根据更新后的隶属关系，更新每个聚类中心：
$$
\mathbf{u}_j = \frac{1}{\lvert c_j \rvert} \sum_{i \in c_j} \mathbf{x}_i
$$

4. 重复以上两个步骤，直到收敛或达到设定的迭代步数为止。

那么，该如何判断终止条件呢？首先当然我们可以设定最大的迭代步数，还有就是聚类中心的变化率等等判断是否已经收敛。

## 3. $k$-means的数学解析

设每个聚类的大小分别为$N_1, \ldots, N_k$，而聚类中心分别为$\mathbf{u_1, \ldots, u_k}$。

如果我们用距离的总平方作为目标函数，也就是：
$$
\min\;\mathbf{J}(\mathbf{u_1, \ldots, u_k}) = \frac{1}{2}\sum_{j=1}^k \sum_{i=1}^N \lVert \mathbf{x_i - u_j} \rVert^2
$$

该函数是关于$\mathbf{u_1, \ldots, u_k}$的凸函数，其驻点为：
$$
\frac{\partial J}{\partial u_j} = \sum_{x_i \in c_j} (x_i - u_j) = 0 \Rightarrow u_j = \frac{1}{N_j}\sum_{x_i \in c_j} x_i
$$

## 4. $k$-means算法存在的一些问题

首先，在更新聚类中心时采用了均值的计算，我们都知道均值的计算__对离群点（outlier）是敏感__的，这容易导致结果不稳定，因此在实务工作中有时候我们页采用计算中位值，这被称为__$k$-medoids聚类__。

此外，初始聚类中心的选择对结果有很大影响，这是因为$k$-means容易陷入局部最优，且目前的算法很难跳出局部最优。这样就要求我们在进行聚类时，应执行多次聚类，然后再考察聚类的结果。

## 5. 小结

作为一种经典的聚类算法，$k$-means聚类具有直观、简单、快速的优点，且在处理大数据时，具有高伸缩性（scalable）和高效（high efficiency），因此被广泛应用，根据其理论特点和实务中的应用来看，其更加适用于聚类类高斯分布的数据簇。但是，我们在应用时是不是应该考察数据是否符合高斯分布呢，答案显然是否定的，且不说很难有合适的方法去实现，即便有，也需要大量的计算，这显然不符合其简单高效的特点。

当然，$k$-means也有其缺陷：

1. 由于在计算聚类中心时，需要计算均值，但某些时候这个均值难以定义，特别是对一些分类（categorical）数据来说，这很难定义。

2. 聚类数目$k$必须是先验的，如果缺乏先验的知识，只能从数据去发现$k$，但这必然存在一定的主观性。

3. 不同的聚类中心的初始值可能会给出不同的聚类结果，有时难以平衡以及对数据进行有效的解读。

4. 不适合发现非凸的聚类，或者聚类簇的大小差别很大的情况。

5. 对噪声和孤立数据敏感。

## $\S$附录1：聚类结果评价的内在标准：轮廓系数（Silhouette coefficient）

Silhouette系数是Peter J. Rousseeuw于1986提出的，评价聚类结果有效性的一个内在标准（internal standard）。

- 首先，我们可以计算样本$i$到类内其他样本的平均距离$a_i$。$a_i$越小，说明样本$i$越应该被聚类到该簇，我们也将$a_i$称为样本$i$的__类内不相似度 (intra-cluster dissimilarity)__:
$$
a_i = \frac{1}{N_i-1}\sum_{j \in c_i} \operatorname{dist}(\mathbf{x}_i, \mathbf{x}_j)
$$
则__聚类簇$c_j$的类内不相似度__可以定义为：
$$
a_{c_j} = \frac{1}{N_j}\sum_{i \in c_j}a_i
$$

- 计算样本$i$到其他类$c_j$的平均距离$b_{ij}$，称为__样本$i$与聚类簇$c_j$的不相似度__。这样我们可以定义__样本$i$的类间不相似度__：
$$
b_i = \min_{j: x_i \not\in c_j} b_{ij}
$$
$b_i$越大，说明样本$i$越不应该属于其他聚类簇。

- 根据样本$i$的类内不相似度$a_i$和类间不相似度$b_i$，我们可以定义样本$i$的轮廓系数为：
$$
s_i = \frac{b_i - a_i}{\max(a_i, b_i)} = \left\{
\begin{array}{ll}
1 - \frac{a_i}{b_i}, & a_i < b_i\\
0, & a_i = b_i\\
\frac{b_i}{a_i} - 1, & a_i > b_i
\end{array}
\right.
$$

- $s_i$接近1,则说明样本$i$聚类合理；$s_i$接近-1,则说明样本$i$更应该分类到另外的簇；若$s_i$近似为0,则说明样本$i$在两个聚类簇的边界上。

- 所有样本的$s_i$的均值称为聚类结果的轮廓系数,是评价该聚类是否合理的其中一个重要度量。

## $\S$附录2：聚类结果评价的其他标准